# BLS Data Query and Cleaning

__This notebook describes ways to query US Bureau of Labor Statistics employment data and clean the data into a tidy vertical format.__

The __Bureau of Labor Statistics__ is a unit of the US Dept. of Labor whose mission is to collect and publish data related to the American economy.

### BLS API
The BLS API v2 requires that you [register (for free)](https://data.bls.gov/registrationEngine/) before you proceed with any API request. Once registered, you must store your API key in a config file or environment variable. This notebook reads the API key from a config file module named __api_key__. 

The API serves data from a POST request with your API key and specific query IDs.


### Finding your query ID
To query BLS data, you must know the specific query ID. Query IDs are specific down to the level of Alabama/Statewide/Total Nonfarm, or deeper. From the [BLS Data site](https://www.bls.gov/data/), click on __One-Screen__ and narrow your search to find the series IDs in the format you are looking for.

### blspandas import
This notebook imports functions from a module called blspandas.py, included in the git repo. The functions in this file are wrappers for many of the functions described by [this article by BD Economics](https://www.bd-econ.com/blsapi.html). 

### Queries
This notebook performs three queries for employment changes between January and June 2020 categorized by: 

1. State
2. Race
3. Industry

After each query and cleaning, the data will be saved as a .csv file and added to a SQLite database.

## Query 1: Changes in Unemployment by State

__BLS Subheader:__ Unemployment

__Database Name:__ Local Area Unemployment Statistics - One-Screen Search

In [7]:
import pandas as pd
import blspandas
import api_key
import requests
import json

# Pull a list of state fips Ids.
bls_fips = blspandas.get_state_fips()

# Create a dictionary of BLS query Ids and States
# Split out to two dicts, one for each half (25) of the 50 states/territories
bls_dict1, bls_dict2 = blspandas.get_bls_id_dicts(bls_fips,'LAUST','0000000000003')

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(api_key.bls_key)

# Query the BLS API using batches - 
query1 = blspandas.query_bls(bls_dict1,key)
query2 = blspandas.query_bls(bls_dict2,key)

# Combine the queries horizontally
combined_queries = pd.concat([query1,query2],axis=1).reset_index().rename(columns={'index': 'Date'})
# Clean the dataframe and melt to vertical format
df = blspandas.clean_bls_data(combined_queries)

df.head()

,Date,State,Pct_Unemployed
0,2019-01-01,Alabama,4.2
1,2019-02-01,Alabama,3.8
2,2019-03-01,Alabama,3.4
3,2019-04-01,Alabama,2.6
4,2019-05-01,Alabama,2.5


Let's write our data to a SQLite database named __blsdata.db__ for convenience.

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///blsdata.db', echo=False)

df.to_sql('Monthly_pct_unemployed', con=engine)
df.to_csv('Monthly percent unemployed.csv')

In [11]:
df.dtypes

Date              datetime64[ns]
State                     object
Pct_Unemployed           float64
dtype: object

### Change in unemployment from January to June 2020

In [17]:
# Convert the Date column back to string,
# find January 1st 2020, 
# only keep State and percent unemployed columns
# Set index to State
jan_emp = df[df['Date'].dt.strftime(date_format='%Y-%m-%d').str.contains('2020-01-01')][['State','Pct_Unemployed']].set_index('State')
jan_emp.head()

,Pct_Unemployed
State,
Alabama,3.2
Alaska,6.6
Arizona,4.6
Arkansas,4.1
California,4.3


In [18]:
# Convert the Date column back to string,
# find June 1st 2020, 
# only keep State and percent unemployed columns
# Set index to State
june_emp = df[df['Date'].dt.strftime(date_format='%Y-%m-%d').str.contains('2020-06-01')][['State','Pct_Unemployed']].set_index('State')
june_emp.head()

,Pct_Unemployed
State,
Alabama,8.0
Alaska,12.3
Arizona,10.3
Arkansas,8.2
California,15.1


Subtract January data from June data

In [20]:
chg_emp = june_emp.subtract(jan_emp)

# Write the dataframe to our SQLite db and a csv file.
chg_emp.to_sql('Chg_employment_state',con=engine)
chg_emp.to_csv('Change in unemployment by State.csv')

In [168]:
chg_emp.head()

,Pct_Unemployed
State,
Alabama,4.8
Alaska,5.7
Arizona,5.7
Arkansas,4.1
California,10.8


__Find the top ten states with highest changes in unemployment__

In [21]:
top_chg_emp = chg_emp.sort_values(by='Pct_Unemployed',ascending=False)[:10]
top_chg_emp

,Pct_Unemployed
State,
Massachusetts,14.1
New Jersey,12.0
Hawaii,11.5
New York,11.5
Nevada,11.4
California,10.8
Michigan,10.6
Illinois,10.6
New Hampshire,8.6


__Now we want to select only these states from our original dataset of monthly unemployment by state__

In [28]:
states = top_chg_emp.index

top_states_df = df[df['State'].isin(states)]
top_states_df.head()

,Date,State,Pct_Unemployed
72,2019-01-01,California,4.8
73,2019-02-01,California,4.5
74,2019-03-01,California,4.5
75,2019-04-01,California,3.8
76,2019-05-01,California,3.6


In [23]:
top_states_df.to_csv('States with highest changes in unemployment.csv',index=False)

## Unemployment and Race
Go back to the bls.gov one-screen finder for [Labor Force Statistics including the National Unemployment Rate](https://www.bls.gov/data/)

In [27]:
import pandas as pd
import blspandas
import api_key
import requests
import json

bls_dict = {
    'LNU04076977': 'White',
    'LNU04076978': 'Black',
    'LNU04076979': 'Asian'
}

# API key in config.py which contains: bls_key = 'key'.
key = '?registrationkey={}'.format(api_key.bls_key)

# Query the BLS API.
query = blspandas.query_bls(bls_dict,key)
query.head()

,White,Black,Asian
2019-01-01,3.9,7.0,3.2
2019-02-01,3.4,7.1,3.1
2019-03-01,3.3,6.6,2.9
2019-04-01,2.8,5.9,2.1
2019-05-01,2.9,5.7,2.3


__Let's find some rolling percent changes for each race__

In [31]:
df = query.copy().reset_index().rename(columns={'index': 'Date'})

df['White_pct_change'] = df.White.pct_change()
df['Black_pct_change'] = df.Black.pct_change()
df['Asian_pct_change'] = df.Asian.pct_change()
df.head()

,Date,White,Black,Asian,White_pct_change,Black_pct_change,Asian_pct_change
0,2019-01-01,3.9,7.0,3.2,NaN,NaN,NaN
1,2019-02-01,3.4,7.1,3.1,-0.128205,0.014286,-0.031250
2,2019-03-01,3.3,6.6,2.9,-0.029412,-0.070423,-0.064516
3,2019-04-01,2.8,5.9,2.1,-0.151515,-0.106061,-0.275862
4,2019-05-01,2.9,5.7,2.3,0.035714,-0.033898,0.095238


In [4]:
df.to_csv('Racial unemployment stats - pct change.csv')

In [51]:
# Melt the dataframe into vertical format.
melted_pct_change = pd.melt(df, id_vars=['Date'],
                   value_vars=df.columns[4:],
                   var_name='Race',
                   value_name='Unemployment Rate Percent Change')

melted_unemp = pd.melt(df, id_vars=['Date'],
                   value_vars=df.columns[1:4],
                   var_name='Race',
                   value_name='Unemployment Rate')

# Merge the two melted dataframes
joined_race_unemp_df = pd.merge(melted_pct_change,melted_unemp,how='inner',on='Date')
# Select only the columns we would like to keep
joined_race_unemp_df = joined_race_unemp_df.loc[:,('Date','Race_y','Unemployment Rate Percent Change','Unemployment Rate')]
joined_race_unemp_df = joined_race_unemp_df.rename(columns={'Race_y':'Race'})
joined_race_unemp_df.head()

,Date,Race,Unemployment Rate Percent Change,Unemployment Rate
0,2019-01-01,White,NaN,3.9
1,2019-01-01,Black,NaN,7.0
2,2019-01-01,Asian,NaN,3.2
3,2019-01-01,White,NaN,3.9
4,2019-01-01,Black,NaN,7.0


In [53]:
joined_race_unemp_df.to_sql('Race_unemp_stats',con=engine)
joined_race_unemp_df.to_csv('Racial Stats unemployment rate and percent change.csv')

# Unemployment by industry

Employment, Hours, and Earnings from the Current Employment Statistics survey (National)

__Goal: dataframe with Date | Employment Ct | Change | Percent Change__

In [54]:
import pandas as pd
import blspandas
import api_key
import requests
import json

bls_dict = {
    'CEU0600000001': 'Goods producing',
    'CEU0700000001': 'Service providing',
    'CEU0800000001': 'Private service providing',
    'CEU1000000001': 'Mining and logging',
    'CEU2000000001': 'Construction',
    'CEU3000000001': 'Manufacturing',
    'CEU3100000001': 'Durable goods',
    'CEU3200000001': 'Nondurable goods',
    'CEU4000000001': 'Trade Transportation and Utilities',
    'CEU4142000001': 'Wholesale Trade',
    'CEU4200000001': 'Retail Trade',
    'CEU4300000001': 'Transportation and Warehousing',
    'CEU4422000001': 'Utilities',
    'CEU5000000001': 'Information',
    'CEU5500000001': 'Financial activities',
    'CEU6000000001': 'Professional and business services',
    'CEU6500000001': 'Education and health services',
    'CEU7000000001': 'Leisure and hospitality',
    'CEU8000000001': 'Other services',
    'CEU9000000001': 'Government' 
}

# API key in config.py which contains: bls_key = 'key'
key = '?registrationkey={}'.format(api_key.bls_key)

# Query the BLS API using batches
query = blspandas.query_bls(bls_dict,key).reset_index().rename(columns={'index':'Date'})

We are going to have three dataframes:

1. Count
2. Change
3. Percent changes

We will perform changes, melt them to vertical format, and combine them.

In [64]:
# Let's prep pure count by melting to our desired format
# pure count
ind_ct = query.copy()
ind_ct.head()

,Date,Goods producing,Service providing,Private service providing,Mining and logging,Construction,Manufacturing,Durable goods,Nondurable goods,Trade Transportation and Utilities,...,Retail Trade,Transportation and Warehousing,Utilities,Information,Financial activities,Professional and business services,Education and health services,Leisure and hospitality,Other services,Government
0,2019-01-01,20539.0,127340.0,104962.0,733.0,7069.0,12737.0,8023.0,4714.0,27598.0,...,15628.1,5582.6,549.7,2799.0,8618.0,20709.0,23724.0,15739.0,5775.0,22378.0
1,2019-02-01,20554.0,128130.0,105337.0,730.0,7062.0,12762.0,8035.0,4727.0,27351.0,...,15429.4,5521.1,547.1,2830.0,8633.0,20841.0,24044.0,15841.0,5797.0,22793.0
2,2019-03-01,20675.0,128684.0,105798.0,731.0,7170.0,12774.0,8039.0,4735.0,27358.0,...,15425.8,5525.3,549.4,2838.0,8658.0,20908.0,24113.0,16090.0,5833.0,22886.0
3,2019-04-01,20884.0,129546.0,106598.0,732.0,7377.0,12775.0,8038.0,4737.0,27425.0,...,15474.7,5525.7,546.5,2832.0,8678.0,21169.0,24206.0,16406.0,5882.0,22948.0
4,2019-05-01,21089.0,130020.0,107164.0,739.0,7540.0,12810.0,8052.0,4758.0,27554.0,...,15540.7,5564.7,548.2,2842.0,8707.0,21239.0,24121.0,16788.0,5913.0,22856.0


In [65]:
melted_ct = pd.melt(ind_ct, id_vars=['Date'],
                   value_vars=ind_ct.columns[1:],
                   var_name='Industry',
                   value_name='Employment')

melted_ct.head()

,Date,Industry,Employment
0,2019-01-01,Goods producing,20539.0
1,2019-02-01,Goods producing,20554.0
2,2019-03-01,Goods producing,20675.0
3,2019-04-01,Goods producing,20884.0
4,2019-05-01,Goods producing,21089.0


__Change down the column / per industry__

In [66]:
# Calculate change down each column
ind_ct_chg = ind_ct.set_index('Date').diff(axis=0).reset_index()
ind_ct_chg.head()

,Date,Goods producing,Service providing,Private service providing,Mining and logging,Construction,Manufacturing,Durable goods,Nondurable goods,Trade Transportation and Utilities,...,Retail Trade,Transportation and Warehousing,Utilities,Information,Financial activities,Professional and business services,Education and health services,Leisure and hospitality,Other services,Government
0,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-01,15.0,790.0,375.0,-3.0,-7.0,25.0,12.0,13.0,-247.0,...,-198.7,-61.5,-2.6,31.0,15.0,132.0,320.0,102.0,22.0,415.0
2,2019-03-01,121.0,554.0,461.0,1.0,108.0,12.0,4.0,8.0,7.0,...,-3.6,4.2,2.3,8.0,25.0,67.0,69.0,249.0,36.0,93.0
3,2019-04-01,209.0,862.0,800.0,1.0,207.0,1.0,-1.0,2.0,67.0,...,48.9,0.4,-2.9,-6.0,20.0,261.0,93.0,316.0,49.0,62.0
4,2019-05-01,205.0,474.0,566.0,7.0,163.0,35.0,14.0,21.0,129.0,...,66.0,39.0,1.7,10.0,29.0,70.0,-85.0,382.0,31.0,-92.0


In [67]:
# Melt to vertical format
melted_ind_ct_chg = pd.melt(ind_ct_chg, id_vars=['Date'],
                   value_vars=ind_ct_chg.columns[1:],
                   var_name='Industry',
                   value_name='Change in Employment')

melted_ind_ct_chg = melted_ind_ct_chg.dropna()

melted_ind_ct_chg.head()

,Date,Industry,Change in Employment
1,2019-02-01,Goods producing,15.0
2,2019-03-01,Goods producing,121.0
3,2019-04-01,Goods producing,209.0
4,2019-05-01,Goods producing,205.0
5,2019-06-01,Goods producing,278.0


__Calculate percent change down the column__ <br>
[Pandas percent change](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pct_change.html)

In [68]:
# Calculate percent change down each column
ind_pct_chg = ind_ct.set_index('Date').pct_change().reset_index()
ind_pct_chg.head()

,Date,Goods producing,Service providing,Private service providing,Mining and logging,Construction,Manufacturing,Durable goods,Nondurable goods,Trade Transportation and Utilities,...,Retail Trade,Transportation and Warehousing,Utilities,Information,Financial activities,Professional and business services,Education and health services,Leisure and hospitality,Other services,Government
0,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-01,0.000730,0.006204,0.003573,-0.004093,-0.000990,0.001963,0.001496,0.002758,-0.008950,...,-0.012714,-0.011016,-0.004730,0.011075,0.001741,0.006374,0.013488,0.006481,0.00381,0.018545
2,2019-03-01,0.005887,0.004324,0.004376,0.001370,0.015293,0.000940,0.000498,0.001692,0.000256,...,-0.000233,0.000761,0.004204,0.002827,0.002896,0.003215,0.002870,0.015719,0.00621,0.004080
3,2019-04-01,0.010109,0.006699,0.007562,0.001368,0.028870,0.000078,-0.000124,0.000422,0.002449,...,0.003170,0.000072,-0.005278,-0.002114,0.002310,0.012483,0.003857,0.019640,0.00840,0.002709
4,2019-05-01,0.009816,0.003659,0.005310,0.009563,0.022096,0.002740,0.001742,0.004433,0.004704,...,0.004265,0.007058,0.003111,0.003531,0.003342,0.003307,-0.003512,0.023284,0.00527,-0.004009


In [70]:
# Melt to vertical format
melted_ind_pct_chg = pd.melt(ind_pct_chg, id_vars=['Date'],
                   value_vars=ind_pct_chg.columns[1:],
                   var_name='Industry',
                   value_name='Percent Change in Employment')

melted_ind_pct_chg['Percent Change in Employment'] = round(melted_ind_pct_chg['Percent Change in Employment'],6)
melted_ind_pct_chg = melted_ind_pct_chg.dropna()
melted_ind_pct_chg.head()

,Date,Industry,Percent Change in Employment
1,2019-02-01,Goods producing,0.000730
2,2019-03-01,Goods producing,0.005887
3,2019-04-01,Goods producing,0.010109
4,2019-05-01,Goods producing,0.009816
5,2019-06-01,Goods producing,0.013182


### Combine the three dataframes into a new dataframe

In [71]:
# Merge the three dataframe
industry_emp_chg = melted_ct.merge(melted_ind_ct_chg, on='Date',).merge(melted_ind_pct_chg, on='Date')
industry_emp_chg.head()

,Date,Industry_x,Employment,Industry_y,Change in Employment,Industry,Percent Change in Employment
0,2019-02-01,Goods producing,20554.0,Goods producing,15.0,Goods producing,0.000730
1,2019-02-01,Goods producing,20554.0,Goods producing,15.0,Service providing,0.006204
2,2019-02-01,Goods producing,20554.0,Goods producing,15.0,Private service providing,0.003573
3,2019-02-01,Goods producing,20554.0,Goods producing,15.0,Mining and logging,-0.004093
4,2019-02-01,Goods producing,20554.0,Goods producing,15.0,Construction,-0.000990


In [72]:
# Choose only the columns we need, removing the unnecessary Date and Industry columns
industry_emp_chg = industry_emp_chg.loc[:,('Date','Industry','Employment','Change in Employment','Percent Change in Employment')]
industry_emp_chg.head()

,Date,Industry,Employment,Change in Employment,Percent Change in Employment
0,2019-02-01,Goods producing,20554.0,15.0,0.000730
1,2019-02-01,Service providing,20554.0,15.0,0.006204
2,2019-02-01,Private service providing,20554.0,15.0,0.003573
3,2019-02-01,Mining and logging,20554.0,15.0,-0.004093
4,2019-02-01,Construction,20554.0,15.0,-0.000990


In [73]:
industry_emp_chg.to_sql('Industry_emp_chg',con=engine)
industry_emp_chg.to_csv('Industry employment changes.csv',index=False)

# Jobs lost to COVID-19 by Industry

In [94]:
from datetime import datetime

jobs_lost = ind_ct.copy()

# Extract Year and Month from the Date column
jobs_lost['Month'] = pd.to_datetime(jobs_lost.Date).dt.month
jobs_lost['Year'] = pd.to_datetime(jobs_lost.Date).dt.year

# Subset January 2020 and June 2020
jobs_lost = jobs_lost[(jobs_lost.Year == 2020) & (jobs_lost.Month.isin([1,6]))]
jobs_lost

,Date,Goods producing,Service providing,Private service providing,Mining and logging,Construction,Manufacturing,Durable goods,Nondurable goods,Trade Transportation and Utilities,...,Utilities,Information,Financial activities,Professional and business services,Education and health services,Leisure and hospitality,Other services,Government,Month,Year
12,2020-01-01,20707.0,129446.0,106849.0,701.0,7240.0,12766.0,8019.0,4747.0,27734.0,...,545.2,2857.0,8762.0,21139.0,24403.0,16092.0,5862.0,22597.0,1,2020
17,2020-06-01,20156.0,118357.0,97204.0,628.0,7359.0,12169.0,7616.0,4553.0,25753.0,...,540.0,2586.0,8650.0,19836.0,22569.0,12556.0,5254.0,21153.0,6,2020


Find the difference between the two rows as the amount of jobs lost.

In [95]:
jobs_lost = jobs_lost.set_index('Date').diff(axis=0).iloc[-1,1:-2]

# Recast to a dataframe
jobs_lost = pd.DataFrame(jobs_lost).reset_index()
# Rename index columnt to Industry
jobs_lost = jobs_lost.rename(columns={'index':'Industry'})
# Rename the date column to Jobs lost
jobs_lost['Jobs lost'] = jobs_lost.iloc[:,1]
# Select only the two columns we need
jobs_lost = jobs_lost.loc[:,('Industry','Jobs lost')]
jobs_lost.head()

,Industry,Jobs lost
0,Service providing,-11089.0
1,Private service providing,-9645.0
2,Mining and logging,-73.0
3,Construction,119.0
4,Manufacturing,-597.0


In [ ]:
jobs_lost.to_sql('Jobs_lost_industry',con=engine)
jobs_lost.to_csv('Jobs lost per industry.csv',index=False)